In [1]:
import nilearn
import brainspace
import numpy as np
import os
import nibabel as nib
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from austin_utils import save_gifti

In [2]:
lab_l = nib.load('HCP/labels/fsLR.32k.L.label.gii').agg_data()
lab_r = nib.load('HCP/labels/fsLR.32k.R.label.gii').agg_data()
labs  = np.concatenate((lab_l, lab_r))

In [3]:
subjects = ["100206", "100307", "100408", "100610", "101006"]
gradient_list = []
for subject in subjects:
    gradient = np.load("HCP/Gradients/%s/%s.mapalign.ses1.diffmap.s02mm.npy" % (subject, subject))
    gradient_list.append(gradient)

gradients_ses1 = np.stack((gradient_list))

gradient_list = []
for subject in subjects:
    gradient = np.load("HCP/Gradients/%s/%s.mapalign.ses2.diffmap.s02mm.npy" % (subject, subject))
    gradient_list.append(gradient)

gradients_ses2 = np.stack((gradient_list))

In [4]:
labeled_grads_ses1 = np.zeros((5, 3, 32492*2))

for subject, subject_map in zip(gradients_ses1, labeled_grads_ses1):
    subject_map[0][labs != 0] = subject[0]
    subject_map[1][labs != 0] = subject[1]
    subject_map[2][labs != 0] = subject[2]


labeled_grads_ses2 = np.zeros((5, 3, 32492*2))

for subject, subject_map in zip(gradients_ses2, labeled_grads_ses2):
    subject_map[0][labs != 0] = subject[0]
    subject_map[1][labs != 0] = subject[1]
    subject_map[2][labs != 0] = subject[2]

In [5]:
sessions = [labeled_grads_ses1, labeled_grads_ses2] #saving each gradient as gifti
hemi_L = "L"
hemi_R = "R"
grads = ["1", "2", "3"]
filenames_L= []
filenames_R = []
gii_paths_L= []
gii_paths_R = []
for session, session_n in zip(sessions, ["1", "2"]):
    for subject, id in zip(session, subjects):
        for gradient_array, grad in zip(subject, grads):
            filename_L = "%s_grad%s_ses%s_%s" % (id, grad, session_n, hemi_L)
            filename_R = "%s_grad%s_ses%s_%s" % (id, grad, session_n, hemi_R)
            save_gifti(gradient_array[:int(len(gradient_array)/2)], filename_L)
            save_gifti(gradient_array[int(len(gradient_array)/2):], filename_R)
            filenames_L.append(filename_L)
            filenames_R.append(filename_R)
            gii_paths_L.append("/Users/VictoriaShevchenko/Documents/PhD/"+ filename_L + ".func.gii")
            gii_paths_R.append("/Users/VictoriaShevchenko/Documents/PhD/"+ filename_R + ".func.gii")

In [6]:
%%capture
# Resampling surface gradient maps to 4k
directory = "/Users/VictoriaShevchenko/Documents/PhD/HCP/resampled_grads/"
import os
for path_L, filename_L in zip(gii_paths_L, filenames_L):
    out_file = directory + filename_L + "4k.func.gii"
    os.system("wb_command -metric-resample %s /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.L.sphere.32k_fs_LR.surf.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.L.sphere.4k_fs_LR.surf.gii BARYCENTRIC %s" %(path_L, out_file))

for path_R, filename_R in zip(gii_paths_R, filenames_R):
    out_file = directory + filename_R + "4k.func.gii"
    os.system("wb_command -metric-resample %s /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.R.sphere.32k_fs_LR.surf.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.R.sphere.4k_fs_LR.surf.gii BARYCENTRIC %s" %(path_R, out_file))

In [7]:
# resampling labels
os.system("wb_command -metric-resample /Users/VictoriaShevchenko/Documents/PhD/HCP/labels/fsLR.32k.L.label.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.L.sphere.32k_fs_LR.surf.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.L.sphere.4k_fs_LR.surf.gii BARYCENTRIC /Users/VictoriaShevchenko/Documents/PhD/HCP/labels/fsLR.4k.L.label_metric.gii")
os.system("wb_command -metric-resample /Users/VictoriaShevchenko/Documents/PhD/HCP/labels/fsLR.32k.R.label.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.R.sphere.32k_fs_LR.surf.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.R.sphere.4k_fs_LR.surf.gii BARYCENTRIC /Users/VictoriaShevchenko/Documents/PhD/HCP/labels/fsLR.4k.R.label_metric.gii")

os.system("wb_command -label-resample /Users/VictoriaShevchenko/Documents/PhD/HCP/labels/fsLR.32k.L.label.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.L.sphere.32k_fs_LR.surf.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.L.sphere.4k_fs_LR.surf.gii BARYCENTRIC /Users/VictoriaShevchenko/Documents/PhD/HCP/labels/fsLR.4k.L.label.gii")
os.system("wb_command -label-resample /Users/VictoriaShevchenko/Documents/PhD/HCP/labels/fsLR.32k.R.label.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.R.sphere.32k_fs_LR.surf.gii /Users/VictoriaShevchenko/Documents/PhD/HCP/spheres/S1200.R.sphere.4k_fs_LR.surf.gii BARYCENTRIC /Users/VictoriaShevchenko/Documents/PhD/HCP/labels/fsLR.4k.R.label.gii")

0

0

0

0